[Anyone Can Learn To Code an LSTM-RNN in Python (Part 1: RNN) - i am trask](http://iamtrask.github.io/2015/11/15/anyone-can-code-lstm/)

加法器的思路：二进制的加法是一个一个二进制位相加，同时会记录一个满二进一的进位，那么训练时，随机找个c=a+b就是一个样本，输入a、b输出c就是整个lstm的预测过程，我们要训练的就是由a、b的二进制向c转换的各种转换矩阵和权重等，也就是我们要设计的神经网络

## 分拆

In [44]:
import copy, numpy as np

In [45]:
np.random.seed(0)

In [46]:
# 定义 sigmoid 函数
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

In [47]:
# 求导数
def sigmoid_output_to_derivative(output):
    return output*(1-output)

In [48]:
# 字典：整形数字-->二进制，存起来不用随时计算，读取更快
# 二进制维度：8，能表达的最大整数：2^8 = 256，即 largest_number
int2binary = {}
binary_dim = 8
largest_number = pow(2, binary_dim)

In [49]:
# 整数转二进制
# 按行展开
binary = np.unpackbits(np.array([range(largest_number)], dtype=np.uint8).T, axis=1)

In [50]:
np.array([range(largest_number)], dtype=np.uint8).T.shape

(256, 1)

In [51]:
binary.shape

(256, 8)

In [52]:
np.array([range(largest_number)], dtype=np.uint8).shape

(1, 256)

In [53]:
np.unpackbits(np.array([range(largest_number)], dtype=np.uint8), axis=0).shape

(8, 256)

In [54]:
np.unpackbits(np.array([range(largest_number)], dtype=np.uint8).T, axis=0).shape

(2048, 1)

In [55]:
# 存储为 dict
for i in range(largest_number):
    int2binary[i] = binary[i]

In [56]:
binary[1]

array([0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [57]:
int2binary[1]

array([0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [58]:
int2binary[200]

array([1, 1, 0, 0, 1, 0, 0, 0], dtype=uint8)

In [59]:
int2binary[200][7]

0

In [60]:
# 设置参数
# alpha 学习速率
# input_dim 输入层向量维度，因为二进制加法输入两个数字，所以是 2；每次喂入两个 bit（每个 bit 八位）
# hidden_dim 隐层向量维度，也就是神经元的个数
# output_dim 输出层向量维度，因为输出 1 个 c 所以是一维
alpha = 0.1
input_dim = 2
hidden_dim = 16
output_dim = 1

In [61]:
# np.random.random 产生的是 [0, 1) 的随机数， 2 * [0, 1) -1 => [-1, 1)
# 输入层 --> 隐藏层权重矩阵：2*16
# 隐藏层 --> 输出层权重矩阵：16*1
# 隐藏层 --> 隐藏层权重矩阵：16*16，不同 timestep 的
# 2x-1 因为 random.random 是从 0 到 1 的随机数，2x-1 就是 -1 到 1
synapse_0 = 2*np.random.random((input_dim, hidden_dim)) - 1
synapse_1 = 2*np.random.random((hidden_dim, output_dim)) - 1
synapse_h = 2*np.random.random((hidden_dim, hidden_dim)) - 1

In [62]:
# 声明三个矩阵更新
synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

In [99]:
 for j in range(10000):
    
    # 随机生成样本
    a_int = np.random.randint(largest_number/2)
    a = int2binary[a_int]
    
    b_int = np.random.randint(largest_number/2)
    b = int2binary[b_int]
    
    c_int = a_int + b_int
    c = int2binary[c_int]
    
    # 保存模型对 c 的预测
    d = np.zeros_like(c)
    
    # 全局误差，观察模型效果
    # 每一轮重新置为 0 
    overallError = 0
    
    # 存储第二层（输出层）的：导数（梯度）×error
    # 可以理解为更新后的 预测值
    layer_2_deltas = list()
    
    # 存储第一层（隐层）的输出值
    # 赋 0 值作为上一个时间的值
    layer_1_values = list()
    # time step 0 没有前一层隐层，所以给 0
    layer_1_values.append(np.zeros(hidden_dim))
    
    # 遍历二进制中的每一位
    # X和y分别是样本的输入和输出的二进制值的第position位，其中X对于每个样本有两个值，分别是a和b对应的第position位
    # 把样本拆成每个二进制位用于训练是因为二进制加法中存在进位标记正好适合利用LSTM的长短期记忆来训练，
    # 每个样本的8个二进制位刚好是一个时间序列
    for position in range(binary_dim):
        X = np.array([[a[binary_dim - position - 1], b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T
        
        # hidder layer (input ~+ prev_hidden)
        # 这里使用的公式是Ct = σ(W0·Xt + Wh·Ct-1)
        # input*(input->hidden) + pre_layer_output * (hidden->hidden)
        # 本层的输入×输入到中间层的权重矩阵 + 上层隐层输出×上层隐层转移到本层隐层的权重矩阵（记忆模块，有多少记住（转移过来）了）
        # ！Magic！！
        layer_1 = sigmoid(np.dot(X, synapse_0) + np.dot(layer_1_values[-1], synapse_h))
        # outpt layer
        # C2 = σ(W1·C1)
        layer_2 = sigmoid(np.dot(layer_1, synapse_1))
        
        # 计算预测值和真实值的误差
        # 每个元素的误差
        layer_2_error = y - layer_2
        
        # 反向传播
        # 开始反向传导的过程，通过如下（式1）公式计算delta(上面提到的公式用在这里)，
        # 并添加到数组layer_2_deltas中，此数组对于每个二进制位(position)有一个值
        # ！！！！可以理解为更新后的 l2，即更新后的预测值
        layer_2_deltas.append((layer_2_error)*sigmoid_output_to_derivative(layer_2))
        
        # 计算累加总误差，用于展示和观察
        # 每次计算的总误差
        overallError += np.abs(layer_2_error[0])

        # 存储预测出来的 position 位的输出值
        # decode estimate so we can print it out
        # layer_2[0][0] 是个纯数字
        # 要么是 0 要么是 1，大于 0.5,1；小于等于 0.5,0
        d[binary_dim - position -1] = np.round(layer_2[0][0])
        
        # store hidden layer so we can use it in the next timestep
        # 存储中间过程生成的隐藏层的值
        layer_1_values.append(copy.deepcopy(layer_1))
        
    # 存储用于下一个时间周期用到的隐层的历史记忆值
    # 先赋空值
    # 每一轮反向传播开始时都要置为 0 
    future_layer_1_delta = np.zeros(hidden_dim)
    
    # 再次遍历二进制的每一位
    # 反向传播
    for position in range(binary_dim):
        
        # 和前面一样取出X的值，不同的是我们从大位开始做更新，因为反向传导是按时序逆着一级一级更新的
        X = np.array([[a[position], b[position]]])
        
        # 取出这一位对应隐层的输出
        layer_1 = layer_1_values[-position-1]
        
        # 取出这一位对应隐层的上一时序的输出
        prev_layer_1 = layer_1_values[-position-2]
        
        # 取出这一位对应输出层的 delta
        # error at output layer
        layer_2_delta = layer_2_deltas[-position-1]
        
        # 反向传导（式2），额外加隐层的 \delta 值
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) + layer_2_delta.dot(synapse_1.T)) *\
                        sigmoid_output_to_derivative(layer_1)
        
        # 累加权重矩阵更新
        # 权重矩阵的偏导等于本层的输出与下一层的 delta 点乘
        # 如式3所示
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        # 对前一时序的隐藏层权重矩阵的更新和上面公式类似，只不过改成前一时序的隐藏层输出与本时序的delta的点乘
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        
        synapse_0_update += X.T.dot(layer_1_delta)
        # 记录下本时序的隐藏层的 delta 用来在下一时序使用
        future_layer_1_delta = layer_1_delta
        
    # 更新权重矩阵
    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha

    # 更新变量归零
    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
        
    # 输出总误差信息
    # 打出第一个和最后一个
    if(j % 9999 == 0):
        print (layer_2_error[0])
        print "Error: " + str(overallError)
        print "Pred:" + str(d)
        print "True:" + str(c)
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x*pow(2, index)
        print str(a_int) + '+' + str(b_int) + '=' + str(out)
        print "-"*10

[-0.00013864]
Error: [ 0.08236254]
Pred:[0 1 1 1 0 0 0 0]
True:[0 1 1 1 0 0 0 0]
5+107=112
----------
[-0.00101233]
Error: [ 0.08482019]
Pred:[0 1 1 0 0 1 1 1]
True:[0 1 1 0 0 1 1 1]
60+43=103
----------


- [反向传导算法 - Ufldl](http://deeplearning.stanford.edu/wiki/index.php/%E5%8F%8D%E5%90%91%E4%BC%A0%E5%AF%BC%E7%AE%97%E6%B3%95)
- ![式1](http://www.shareditor.com/uploads/media/my-context/0001/01/5efdad7ad8aeeee394eb2ce5a058582ff4533c61.png)
- ![式2](http://www.shareditor.com/uploads/media/my-context/0001/01/d1ed3d1dd569961b2541d89fe1675f95b1ddbb12.png)
- ![式3](http://www.shareditor.com/uploads/media/my-context/0001/01/2060000a147f56f518d812c48b225e0932ffd294.png)

## 完整

In [7]:
import copy, numpy as np
np.random.seed(0)

# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)


# training dataset generation
int2binary = {}
binary_dim = 8

largest_number = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([range(largest_number)],dtype=np.uint8).T,axis=1)
for i in range(largest_number):
    int2binary[i] = binary[i]


# input variables
alpha = 0.1
input_dim = 2
hidden_dim = 16
output_dim = 1


# initialize neural network weights
synapse_0 = 2*np.random.random((input_dim,hidden_dim)) - 1
synapse_1 = 2*np.random.random((hidden_dim,output_dim)) - 1
synapse_h = 2*np.random.random((hidden_dim,hidden_dim)) - 1

synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

# training logic
for j in range(10000):
    
    # generate a simple addition problem (a + b = c)
    a_int = np.random.randint(largest_number/2) # int version
    a = int2binary[a_int] # binary encoding

    b_int = np.random.randint(largest_number/2) # int version
    b = int2binary[b_int] # binary encoding

    # true answer
    c_int = a_int + b_int
    c = int2binary[c_int]
    
    # where we'll store our best guess (binary encoded)
    d = np.zeros_like(c)

    overallError = 0
    
    layer_2_deltas = list()
    #print layer_2_deltas
    layer_1_values = list()
    #print layer_1_values
    layer_1_values.append(np.zeros(hidden_dim))
    #print layer_1_values
    # moving along the positions in the binary encoding
    for position in range(binary_dim):
        
        # generate input and output
        X = np.array([[a[binary_dim - position - 1],b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T

        # hidden layer (input ~+ prev_hidden)
        layer_1 = sigmoid(np.dot(X,synapse_0) + np.dot(layer_1_values[-1],synapse_h))

        # output layer (new binary representation)
        layer_2 = sigmoid(np.dot(layer_1,synapse_1))

        # did we miss?... if so, by how much?
        layer_2_error = y - layer_2
        layer_2_deltas.append((layer_2_error)*sigmoid_output_to_derivative(layer_2))
        overallError += np.abs(layer_2_error[0])
    
        # decode estimate so we can print it out
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        
        # store hidden layer so we can use it in the next timestep
        layer_1_values.append(copy.deepcopy(layer_1))
    
    future_layer_1_delta = np.zeros(hidden_dim)
    
    for position in range(binary_dim):
        
        X = np.array([[a[position],b[position]]])
        layer_1 = layer_1_values[-position-1]
        prev_layer_1 = layer_1_values[-position-2]
        
        # error at output layer
        layer_2_delta = layer_2_deltas[-position-1]
        # error at hidden layer
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) + layer_2_delta.dot(synapse_1.T)) * sigmoid_output_to_derivative(layer_1)

        # let's update all our weights so we can try again
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)
        
        future_layer_1_delta = layer_1_delta
    

    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha    

    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
    
    # print out progress
    if(j % 1000 == 0):
        print "Error:" + str(overallError)
        print "Pred:" + str(d)
        print "True:" + str(c)
        out = 0
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        print str(a_int) + " + " + str(b_int) + " = " + str(out)
        print "------------"

        

Error:[ 3.45638663]
Pred:[0 0 0 0 0 0 0 1]
True:[0 1 0 0 0 1 0 1]
9 + 60 = 1
------------
Error:[ 3.63389116]
Pred:[1 1 1 1 1 1 1 1]
True:[0 0 1 1 1 1 1 1]
28 + 35 = 255
------------
Error:[ 3.91366595]
Pred:[0 1 0 0 1 0 0 0]
True:[1 0 1 0 0 0 0 0]
116 + 44 = 72
------------
Error:[ 3.72191702]
Pred:[1 1 0 1 1 1 1 1]
True:[0 1 0 0 1 1 0 1]
4 + 73 = 223
------------
Error:[ 3.5852713]
Pred:[0 0 0 0 1 0 0 0]
True:[0 1 0 1 0 0 1 0]
71 + 11 = 8
------------
Error:[ 2.53352328]
Pred:[1 0 1 0 0 0 1 0]
True:[1 1 0 0 0 0 1 0]
81 + 113 = 162
------------
Error:[ 0.57691441]
Pred:[0 1 0 1 0 0 0 1]
True:[0 1 0 1 0 0 0 1]
81 + 0 = 81
------------
Error:[ 1.42589952]
Pred:[1 0 0 0 0 0 0 1]
True:[1 0 0 0 0 0 0 1]
4 + 125 = 129
------------
Error:[ 0.47477457]
Pred:[0 0 1 1 1 0 0 0]
True:[0 0 1 1 1 0 0 0]
39 + 17 = 56
------------
Error:[ 0.21595037]
Pred:[0 0 0 0 1 1 1 0]
True:[0 0 0 0 1 1 1 0]
11 + 3 = 14
------------
